# Flights' delay problem

<img src="https://drive.google.com/uc?export=view&id=1xrvWM5Nfy38NgrWtKjf9dxZHgr-zDHHL" width=500>



Is the original order the most convenient now?
Can we reduce the overall costs condidering that no flight can arrive earlier than its *expected arrival time* (ETA)?


In [15]:
import gurobipy as gb
import numpy as np
import matplotlib.pyplot as plt

In [42]:
class Flight:

  def __init__(self, index, airline, cost_coefficient, cost_kind = "quad"):

    self.name = "F" + airline.name + str(index)
    self.airline = airline
    self.airline.flights.append(self)
    self.index = index
    self.cost = cost_coefficient
    self.eta = index
    self.sol = None
    if cost_kind == "quad":
      cost_fun = lambda delay: 0 if delay<0 else self.cost * delay**2
    else:
      cost_fun = lambda delay: 0 if delay<0 else self.cost * delay
    self.costVect = np.array([cost_fun(t - self.eta) for t in np.arange(0,40,2)])

  def __repr__(self):
    return self.name

  def __str__(self):
    return self.name

In [52]:
class Airline:
  def __init__(self, name):
    self.name = name
    self.initial_costs = None
    self.final_costs = None
    self.reduction = None
    self.reduction_percentage = None
    self.flights = []

  def compute_costs(self):
    self.initial_costs = sum([flight.costVect[flight.index] for flight in self.flights])
    self.final_costs = sum([flight.costVect[flight.sol] for flight in self.flights])
    self.reduction = self.final_costs- self.initial_costs
    self.reduction_percentage =  self.reduction/self.initial_costs

In [53]:
def initial_costs(flights):
  return sum([flight.costVect[flight.eta] for flight in flights])

def make_flights(airlines_names, cost_coefficients, cost_kind="quad", show=False):
  airlines = [Airline(air_name) for air_name in np.unique(airlines_names)]
  airlines_dict = dict(zip([air.name for air in airlines], airlines))
  flights = []
  for i in range(len(airlines_names)):
    flights.append(Flight(i, airlines_dict[airlines_names[i]], cost_coefficients[i], cost_kind))
  if show: 
    for flight in flights:
      print(flight, flight.airline, flight.index, flight.cost, flight.eta)
  return flights, airlines

In [54]:
from gurobipy import GRB, quicksum


class Mincost:
  def __init__(self, flights, airlines, slots, nnb=False):
    self.p = gb.Model()
    self.p.modelSense = GRB.MAXIMIZE
    self.airlines = airlines
    self.p.setParam('OutputFlag', 0) # quite logs
    self.x = self.p.addVars([(flight.index, j) for j in slots for flight in flights], vtype=GRB.BINARY)
    self.nnb = nnb
    self.reduction = None


  def set_constraints(self):

    for flight in flights:
      self.p.addConstr(
          quicksum(self.x[flight.index, j] for j in slots) == 1
      )
      self.p.addConstr(
          quicksum(self.x[flight.index, j] for j in slots if new_slot_time[j] < flight.eta) == 0
      )

    for j in slots:
      self.p.addConstr(
          quicksum(self.x[flight.index, j] for flight in flights) <=1
      )

    if self.nnb:
      for air in self.airlines:
        self.p.addConstr(
            quicksum(self.x[flight.index,j]*flight.costVect[j] for flight in air.flights for j in slots ) <=  sum([flight.costVect[flight.index] for flight in air.flights])
        )

  def set_objective(self):

    self.p.setObjective(
        quicksum( self.x[flight.index, j]*flight.costVect[j] for flight in flights for j in slots)
    )

  def solve(self):
    self.set_constraints()
    self.set_objective()
    self.p.optimize()

    for j in slots:
      for flight in flights:
        if round(self.x[flight.index, j].x) == 1:
          flight.sol = j
          print(flight, flight.cost, j, flight.airline, j*2 - flight.eta) 

    total_init = sum([flight.costVect[flight.index] for flight in flights])
    total_final = self.p.objVal
    total_reduction = (total_init - total_final)/total_init

    print("\nininital cost:", total_init, )
    print("final cost:", total_final)
    print("reduction", total_reduction, "\n")

    airs = []
    for air in self.airlines:
      air.compute_costs()

    for air in airs:
      print(air.name, air.initial_costs, air.final_costs)
    self.reduction = total_reduction

In [55]:
cost_coefficients = np.array([8,1,6,5,10,7,4,2,10,3,21,9,11,2,15])

airlines_name = ["A","C","B","A","B","A","B","C","B","C","A","A","B","C","A"]

num_flights = len(airlines_name)

slots = range(num_flights)
new_slot_time = np.array(np.arange(0,num_flights*2,2))
print(new_slot_time)

flights, airline_list = make_flights(airlines_name, cost_coefficients)

[ 0  2  4  6  8 10 12 14 16 18 20 22 24 26 28]


In [56]:
min_cost = Mincost(flights, airline_list, slots)
min_cost.solve()
nnb = Mincost(flights, airline_list, slots, nnb = True)
nnb.solve()

FA0 8 0 <__main__.Airline object at 0x7f847d334d00> 0
FC1 1 1 <__main__.Airline object at 0x7f847d334ca0> 1
FA3 5 2 <__main__.Airline object at 0x7f847d334d00> 1
FB6 4 3 <__main__.Airline object at 0x7f847d334160> 0
FC7 2 4 <__main__.Airline object at 0x7f847d334ca0> 1
FC9 3 5 <__main__.Airline object at 0x7f847d334ca0> 1
FB12 11 6 <__main__.Airline object at 0x7f847d334160> 0
FC13 2 7 <__main__.Airline object at 0x7f847d334ca0> 1
FA14 15 8 <__main__.Airline object at 0x7f847d334d00> 2
FA11 9 9 <__main__.Airline object at 0x7f847d334d00> 7
FA5 7 10 <__main__.Airline object at 0x7f847d334d00> 15
FB2 6 11 <__main__.Airline object at 0x7f847d334160> 20
FB8 10 12 <__main__.Airline object at 0x7f847d334160> 16
FB4 10 13 <__main__.Airline object at 0x7f847d334160> 22
FA10 21 14 <__main__.Airline object at 0x7f847d334d00> 18

ininital cost: 9581
final cost: 18693.0
reduction -0.951048951048951 

FA0 8 0 <__main__.Airline object at 0x7f847d334d00> 0
FC1 1 1 <__main__.Airline object at 0x7f847d

#Change schedule